In [1]:
%serialconnect

Connecting to Serial /dev/ttyUSB0 baud=115200 
Ready.


In [3]:
# Put the phone on hotspot mode and connect to it
import network, usocket

si = network.WLAN(network.STA_IF)
macaddress = "".join("{:02x}".format(x)  for x in si.config("mac"))
si.active(True)
si.connect(b'BV6000', "beckaaaa")
while not si.isconnected():
    pass
bvip = si.ifconfig()[2]
print(bvip)

> auth (b0)
> assoc (0)
> run (10)
192.168.43.1


In [25]:
# Turn XCSoar on and set device to Port(Port: TCP Port, TCP Port: 2000, Driver: BlueFly Vario) 
s = usocket.socket(usocket.AF_INET, usocket.SOCK_STREAM)
s.connect(usocket.getaddrinfo(bvip, 2000)[0][-1])

In [2]:
# This should appear in the monitor page
s.send("hi there\r\n")

Traceback (most recent call last):
  File "<stdin>", line 2, in <module>
NameError: name 's' is not defined


In [6]:
# XCSoar responds to GPRMC words for GPS fixing
import time, math
baseepoch = time.mktime((2018,1,1,12,0,0,0,0)) # midday of new year

lat0 = 53.41
lng0 = -1.5
earthrad = 6378137
latfac = 360/(2*math.pi*earthrad)
lngfac = latfac/math.cos(math.radians(lat0))

def getrmc(dt, dx, dy):
    ts = time.localtime(baseepoch+int(dt))
    lat, lng = lat0+dy*latfac, lng0 + dx*lngfac
    gt = "%02d%02d%02.3f" % (ts[3], ts[4], ts[5]+(dt%1))
    gpos = "%02d%02.4f,N,%03d%02.4f,%s" % (lat, (lat*60)%60, abs(lng), (abs(lng)*60)%60, "W" if lng<0 else "E")
    gv = "%03.1f,%03.2f" % (0.24, 233.51) # speed knots, course (disregarded)
    gd = "%02d%02d%02d" % (ts[2], ts[1], ts[0]%100)
    g = "GPRMC,%s,A,%s,%s,%s,,,A"%(gt, gpos, gv, gd)
    cs = 0
    for c in g:
        cs ^= ord(c)
    g = "$%s*%02X\r\n" % (g, cs)
    return g

def getprs(alt):
    # p = 101325 * (1 - 2.25577e-5*h)**5.25588
    p = 101300 - 11.45043*alt
    return "PRS %X\r\n"%int(p)


In [22]:
print(60*2*3.14/speed)


7.536


In [29]:
# put out a turning case
rad = 60
speed = 20
tstep = 0.5
climb = 2
vx = 3
h = 1
for i in range(h*100, (h+1)*100):
    dt = i*tstep
    ang = math.radians(dt*speed/(math.pi*2*rad)*360)
    s.send(getrmc(dt, vx*dt + math.cos(ang)*rad, math.sin(ang)*rad))
    s.send(getprs(500+dt*climb))
    time.sleep(tstep)
print("done")

..........done


In [95]:
h = 1000
p = 1013-- * (1 - 2.25577e-5*h)**5.25588 
print(h, p)

1000 89874.57


In [94]:
print((101325-89874.57)/(101325-79495.19))

0.5245319


In [99]:
print("%X"%10000.9)

Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
TypeError: can't convert float to int


In [164]:
while True:
    if u.any():
        k = u.read()
        s.send(k)
        print(k)

b'6,223.05,060318,,,A*79\r\n$GPVTG,223.05,T,,M,0.16,N,0.30,K,A*3,1.23,0.90,0.84*05\r\n$GPRMC,124708.000,A,5324.2196,N,00259.0736,,49.5,M,,*7C\r\n$GPGSA,A,3,15,24,20,12,28,10,13,19,,,,,1.23,0.K,A*3F\r\n$GPGGA,124709.000,5324.2195,N,00259.0736,W,1,08,0.909,222.93,060318,,,A*7A\r\n$GPVTG,222.93,T,,M,0.19,N,0.34,K,A*3,1.23,0.90,0.84*05\r\n$GPRMC,124709.000,A,5324.2195,N,00259.079,W,0.15,238.06,060318,,A*71\r\n$GPVTG,238.06,T'
b'3,19,17,,,,1.20,K,A*3C\r\n$GPGGA,124736.000,5324.2195,N,00259.0740,W,1,09,0.85,117.5,M,49.5,M,,*77\r\n$GPGSA,A,3,15,24,20,12'
b',0.17,N,0.31,K,A,,,1.20,0.85,0.84*04\r\n$GPRMC,124736.000,A,5324.2195,N,00259.0740,W,0.17,263.17,060318,,,A*70\r\n$GPVTG,263'
b'.17,T'
b'3,19,17,,,,1.20,K,A*38\r\n$GPGGA,124737.000,5324.2195,N,00259.0740,W,1,09,0.85,117.5,M,49.5,M,,*76\r\n$GPGSA,A,3,15,24,20,12'
b'313,40,28,23,047,,,1.20,0.85,0.84*04\r\n$GPGSV,3,1,11,15,66,179,16,24,59,267,44,13,41,135,18,17,27,093,25*79\r\n$GPGSV,3,2,1'
b'5324.2195,N,00258,23,047,30,20,19,216,23,19,19,116,

Traceback (most recent call last):
  File "<stdin>", line 2, in <module>
KeyboardInterrupt: 


In [20]:
import time
g = "$GPGGA,124739.000,5324.2195,N,00259.0743,W,1,09,0.85,117.5,M,49.5,M,,*7B\r\n"
f = "$GPRMC,124739.000,A,5324.2195,N,00259.0743,W,0.24,233.51,060318,,,A*7B\r\n"
for i in range(20):
    #s.send(g)
    s.send(f)
    time.sleep(0.5)

..

In [52]:
g = "$GPGGA,064306.590,5323.7234,N,00258.3325,W,  2, 10, 1.00,38.3,M,49.4,M,0000,0000*4A"
s = 0
for c in g[1:-3]:
    s ^= ord(c)
print(hex(s))

0x4a


In [82]:
def getg(dt):
    t = 3701.9 + dt
    x, y = 53.41, 2.9+dt*0.001
    alt = 200+dt*0.1
    timestamp = "%02d%02d%02.3f" % (t/3600, (t/60)%60, t%60)
    pos = "%02d%02.4f,N,%03d%02.4f,W" % (x, (x*60)%60, y, (y*60)%60)
    g = "GPGGA,%s,%s,2,5,1.57,%.1f,M,49.4,M,,"%(timestamp, pos, alt)
    cs = 0
    for c in g:
        cs ^= ord(c)
    g = "$%s*%02X\r\n" % (g, cs)
    return g

In [76]:
print(getg(10))
print(getg(11))
#$GPGGA,081158.400,4837.7021,N,00806.2928,E,2,5,1.57,113.9,M,47.9,M,,*5B


$GPGGA,010151.900,5324.6001,N,00254.6000,W,2,10,1.00,200.9,M,49.4,M,0000,0000*70

$GPGGA,010152.900,5324.6001,N,00254.6600,W,2,10,1.00,201.0,M,49.4,M,0000,0000*7D



In [83]:
import time
for i in range(100):
    s.send(getg(i))
    time.sleep(1)
    

....................

In [102]:
g = "$GPGGA,081158.400,4837.7021,N,00806.2928,E,2,5,1.57,113.9,M,47.9,M,,*5B\n"
s.send(g)

In [125]:
h = s.read(1)
print(h)


b'T'
